In [1]:
ballot = """H.J.R.A No. A99
A JOINT RESOLUTION
proposing a constitutional amendment authorizing the legislature
to exempt from ad valorem taxation tangible personal property
consisting of animal feed held by the owner of the property for sale
at retail.
BE IT RESOLVED BY THE LEGISLATURE OF THE STATE OF TEXAS:
SECTIONA1.AAArticle VIII, Texas Constitution, is amended by
adding Section 1-s to read as follows:
Sec.A1-s.AA(a) The legislature by general law may exempt
from ad valorem taxation tangible personal property consisting of
animal feed held by the owner of the property for sale at retail.
(b)AAThe legislature by general law may provide additional
eligibility requirements for the exemption authorized by this
section.
SECTIONA2.AAThis proposed constitutional amendment shall be
submitted to the voters at an election to be held November 4, 2025.
The ballot shall be printed to permit voting for or against the
proposition: "The constitutional amendment authorizing the
legislature to exempt from ad valorem taxation tangible personal
property consisting of animal feed held by the owner of the property
for sale at retail."
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1______________________________ ______________________________
AAAAPresident of the Senate Speaker of the HouseAAAAAA
I certify that H.J.R. No. 99 was passed by the House on April
28, 2025, by the following vote:AAYeas 102, Nays 5, 28 present, not
voting.
______________________________
Chief Clerk of the HouseAAA
I certify that H.J.R. No. 99 was passed by the Senate on May
9, 2025, by the following vote:AAYeas 30, Nays 1.
______________________________
Secretary of the SenateAAAA
RECEIVED:AA_____________________
AAAAAAAAAAAAAAAAAAAADateAAAAAAAAA
AAAAAAAAAAA_____________________
AAAAAAAAAAAAASecretary of StateAAAAAAA
H.J.R.ANo.A99
2"""

In [6]:
"""
Simple Ballot Measure Analysis Tools

Three approaches to help voters understand ballot measures:
1. Plain language summary
2. Debate between voters (FOR vs AGAINST format)
3. Steelman networks (strongest arguments FOR and AGAINST with critiques)
4. Comprehension questions (5 True/False questions based on summary)

Results are saved to a CSV file for use in Qualtrics.
"""

from typing import Dict, List, Any, Tuple
from litellm import completion
from plurals.agent import Agent
from plurals.deliberation import Debate, Graph, Moderator
from dotenv import load_dotenv
import csv
import json
from datetime import datetime

load_dotenv("../src/.env")


# ============================================================================
# APPROACH 1: Plain Language Summary
# ============================================================================

def plain_summary(ballot_text: str, model: str = "gpt-4o") -> str:
    """
    Generate a straightforward summary of the ballot measure

    Args:
        ballot_text: Full ballot measure text
        model: LLM model to use

    Returns:
        Plain language summary (~300 words)
    """

    prompt = f"""
    Summarize this ballot measure in plain language that any voter can understand.

    Use this EXACT format with these three sections (no other headings, no bullet points):

    BACKGROUND
    [Provide context about why this measure exists and what problem it addresses]

    BALLOT MEASURE
    [Explain in detail what the measure does, who would be affected, and key provisions]

    WHAT A YES AND NO VOTE MEANS
    [Clearly explain what happens if voters vote YES and what happens if voters vote NO]

    Use approximately 300 words total across all three sections.
    Write in paragraph form only - no bullet points, no lists.

    BALLOT MEASURE:
    {ballot_text}
    """

    response = completion(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content


# ============================================================================
# APPROACH 2: Voter Debate (FOR vs AGAINST)
# ============================================================================

def voter_debate(ballot_text: str, model: str = "gpt-4o", cycles: int = 4) -> str:
    """
    Create a debate between a voter FOR and voter AGAINST the measure

    Args:
        ballot_text: Full ballot measure text
        model: LLM model to use
        cycles: Number of back-and-forth exchanges

    Returns:
        Formatted debate string with "Voter 1 [FOR]:" and "Voter 2 [AGAINST]:" labels
        Uses <br><br> for Qualtrics compatibility
    """

    task = f"""
    You are a voter analyzing this ballot measure. State your position and reasoning.

    Keep your response to 150 words maximum.

    BALLOT MEASURE:
    {ballot_text}
    """

    addtl_inst = "Write compellingly and convincingly, using plain language. Imagine you are on Reddit. Highlight non-obvious points a regular voter is likely to miss. Avoid cliches. Do not repeat points already made. Remember to write compellingly and convincingly. Each response should be just 50 words. Don't write to much."

    # Create voter agents
    voter_for = Agent(
        system_instructions="You are a voter who supports this ballot measure. Explain why you're voting FOR it.",
        model=model,
        task=f"Argue FOR this measure: {ballot_text}. {addtl_inst}"
    )

    voter_against = Agent(
        system_instructions="You are a voter who opposes this ballot measure. Explain why you're voting AGAINST it.",
        model=model,
        task=f"Argue AGAINST this measure: {ballot_text}. Each response should be 50 words. {addtl_inst}."
    )

    # Run debate without moderator
    debate = Debate(
        agents=[voter_for, voter_against],
        combination_instructions="debate",
        cycles=cycles
    )

    debate.process()

    # Format responses with proper labels for Qualtrics
    formatted_debate = []
    for i, response in enumerate(debate.responses):
        # Remove "[Debater 1]" and "[Debater 2]" prefixes if present
        clean_response = response.replace("[Debater 1] ", "").replace("[Debater 2] ", "")

        if i % 2 == 0:
            formatted_debate.append(f"Voter 1 [FOR]: {clean_response}")
        else:
            formatted_debate.append(f"Voter 2 [AGAINST]: {clean_response}")

    return "<br><br>".join(formatted_debate)


# ============================================================================
# APPROACH 3: Steelman Networks
# ============================================================================

def steelman_for(ballot_text: str, model: str = "gpt-4o") -> str:
    """
    Generate strongest arguments FOR the measure using a star network

    Star structure: 3 agents submit pro arguments → 1 critic → 1 moderator

    Args:
        ballot_text: Full ballot measure text
        model: LLM model to use

    Returns:
        String with top 3 pro arguments and their counters
    """

    # Task for pro-argument agents
    pro_task = f"""
    Generate the strongest possible argument FOR this ballot measure.

    Focus on:
    - Benefits and positive impacts
    - Why this is needed now
    - Evidence supporting the measure

    Keep response to 100 words.

    BALLOT MEASURE:
    {ballot_text}
    """

    # Create 3 pro-argument agents
    pro_agents = [
        Agent(
            system_instructions=f"You are voter {i+1} who strongly supports this measure. Provide your strongest argument FOR it.",
            model=model
        )
        for i in range(3)
    ]

    # Create critic agent
    critic = Agent(
        system_instructions="You are a critical analyst. For each pro argument you see, provide a thoughtful counter-argument.",
        model=model
    )

    # Create moderator with Qualtrics-compatible formatting
    moderator = Moderator(
        system_instructions="You synthesize debates by identifying the strongest arguments and their counters.",
        model=model,
        combination_instructions="""
        Based on the pro arguments and critiques you see:

        List the top 3 arguments FOR the measure, each followed by its main counter-argument.

        Format as (use <br><br> for line breaks):

        PRO ARGUMENT 1: [argument]<br><br>COUNTER: [counter-argument]<br><br>

        PRO ARGUMENT 2: [argument]<br><br>COUNTER: [counter-argument]<br><br>

        PRO ARGUMENT 3: [argument]<br><br>COUNTER: [counter-argument]

        Previous responses: ${previous_responses}
        """
    )

    # Create star network: all pro agents → critic → moderator
    agents_dict = {
        'pro1': pro_agents[0],
        'pro2': pro_agents[1],
        'pro3': pro_agents[2],
        'critic': critic,
        'moderator': moderator
    }

    edges = [
        ('pro1', 'critic'),
        ('pro2', 'critic'),
        ('pro3', 'critic'),
        ('critic', 'moderator')
    ]

    graph = Graph(
        agents=agents_dict,
        edges=edges,
        task=pro_task,
        combination_instructions="default"
    )

    graph.process()

    return graph.final_response


def steelman_against(ballot_text: str, model: str = "gpt-4o") -> str:
    """
    Generate strongest arguments AGAINST the measure using a star network

    Star structure: 3 agents submit con arguments → 1 critic → 1 moderator

    Args:
        ballot_text: Full ballot measure text
        model: LLM model to use

    Returns:
        String with top 3 con arguments and their counters
    """

    # Task for con-argument agents
    con_task = f"""
    Generate the strongest possible argument AGAINST this ballot measure.

    Focus on:
    - Costs and negative impacts
    - Problems with the measure
    - Evidence opposing the measure

    Keep response to 100 words.

    BALLOT MEASURE:
    {ballot_text}
    """

    # Create 3 con-argument agents
    con_agents = [
        Agent(
            system_instructions=f"You are voter {i+1} who strongly opposes this measure. Provide your strongest argument AGAINST it.",
            model=model
        )
        for i in range(3)
    ]

    # Create critic agent
    critic = Agent(
        system_instructions="You are a critical analyst. For each con argument you see, provide a thoughtful counter-argument.",
        model=model
    )

    # Create moderator with Qualtrics-compatible formatting
    moderator = Moderator(
        system_instructions="You synthesize debates by identifying the strongest arguments and their counters.",
        model=model,
        combination_instructions="""
        Based on the con arguments and critiques you see:

        List the top 3 arguments AGAINST the measure, each followed by its main counter-argument.

        Format as (use <br><br> for line breaks):

        CON ARGUMENT 1: [argument]<br><br>COUNTER: [counter-argument]<br><br>

        CON ARGUMENT 2: [argument]<br><br>COUNTER: [counter-argument]<br><br>

        CON ARGUMENT 3: [argument]<br><br>COUNTER: [counter-argument]

        Previous responses: ${previous_responses}
        """
    )

    # Create star network: all con agents → critic → moderator
    agents_dict = {
        'con1': con_agents[0],
        'con2': con_agents[1],
        'con3': con_agents[2],
        'critic': critic,
        'moderator': moderator
    }

    edges = [
        ('con1', 'critic'),
        ('con2', 'critic'),
        ('con3', 'critic'),
        ('critic', 'moderator')
    ]

    graph = Graph(
        agents=agents_dict,
        edges=edges,
        task=con_task,
        combination_instructions="default"
    )

    graph.process()

    return graph.final_response


# ============================================================================
# APPROACH 4: Comprehension Questions
# ============================================================================

def generate_comprehension_questions(summary: str, model: str = "gpt-4o") -> Dict[str, str]:
    """
    Generate 5 True/False comprehension questions about the ballot measure based on the summary

    Args:
        summary: Plain language summary of the ballot measure
        model: LLM model to use

    Returns:
        Dictionary with questions (TF1-TF5) and answers (TF1_ans-TF5_ans)
    """

    prompt = f"""
    Based on the following ballot measure summary, generate exactly 5 True/False comprehension questions.

    Requirements:
    - Questions MUST test understanding of CORE PROVISIONS and IMPORTANT DETAILS of the measure
    - Focus on: what the measure does, who is affected, key dollar amounts, voting thresholds, implementation details
    - DO NOT ask peripheral or trivial details
    - Mix of true and false answers (not all true or all false)
    - Questions should be clear and unambiguous
    - Answers should be definitively true or false based on the summary

    Return your response as a JSON object with this exact structure:
    {{
        "questions": [
            {{"question": "Question 1 text", "answer": "True"}},
            {{"question": "Question 2 text", "answer": "False"}},
            {{"question": "Question 3 text", "answer": "True"}},
            {{"question": "Question 4 text", "answer": "False"}},
            {{"question": "Question 5 text", "answer": "True"}}
        ]
    }}

    BALLOT MEASURE SUMMARY:
    {summary}
    """

    response = completion(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        response_format={"type": "json_object"}
    )

    # Parse JSON response
    questions_data = json.loads(response.choices[0].message.content)

    # Format for CSV output
    result = {}
    for i, item in enumerate(questions_data["questions"][:5], 1):  # Ensure only 5 questions
        result[f"TF{i}"] = item["question"]
        result[f"TF{i}_ans"] = item["answer"]

    return result


# ============================================================================
# Main Analysis and CSV Export
# ============================================================================

def analyze_ballot_measure(ballot_text: str, model: str = "gpt-4o") -> Dict[str, str]:
    """
    Run all analysis approaches including comprehension questions

    Args:
        ballot_text: Full ballot measure text
        model: LLM model to use

    Returns:
        Dictionary with results from all approaches
    """

    print("Running approach 1: Plain summary...")
    summary = plain_summary(ballot_text, model)

    print("Running approach 2: Voter debate...")
    debate_text = voter_debate(ballot_text, model, cycles= 4)

    print("Running approach 3a: Steelman FOR...")
    steelman_for_text = steelman_for(ballot_text, model)

    print("Running approach 3b: Steelman AGAINST...")
    steelman_against_text = steelman_against(ballot_text, model)

    print("Running approach 4: Comprehension questions...")
    comprehension = generate_comprehension_questions(summary, model)

    # Combine all results
    results = {
        "plain_summary": summary,
        "steelman_for": steelman_for_text,
        "steelman_against": steelman_against_text,
        "debate": debate_text
    }

    # Add comprehension questions and answers
    results.update(comprehension)

    return results


def save_to_csv(results: Dict[str, str], filename: str = None) -> str:
    """
    Save analysis results to a CSV file

    Args:
        results: Dictionary with analysis results
        filename: Optional filename. If None, generates timestamp-based filename

    Returns:
        Filename of saved CSV
    """

    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"ballot_analysis_{timestamp}.csv"

    # Define column order
    fieldnames = [
        'plain_summary',
        'steelman_for',
        'steelman_against',
        'debate',
        'TF1', 'TF1_ans',
        'TF2', 'TF2_ans',
        'TF3', 'TF3_ans',
        'TF4', 'TF4_ans',
        'TF5', 'TF5_ans'
    ]

    # Write to CSV
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        writer.writerow(results)

    print(f"\nResults saved to: {filename}")
    return filename


def analyze_and_save(ballot_text: str, model: str = "gpt-4o", filename: str = None) -> str:
    """
    Complete workflow: analyze ballot measure and save to CSV

    Args:
        ballot_text: Full ballot measure text
        model: LLM model to use
        filename: Optional CSV filename

    Returns:
        Filename of saved CSV
    """

    results = analyze_ballot_measure(ballot_text, model)
    csv_filename = save_to_csv(results, filename)

    return csv_filename


# Example usage
if __name__ == "__main__":

    # Run analysis and save to CSV
    csv_file = analyze_and_save(ballot, filename="ballot_analysis.csv")

    # Also display results to terminal (converting <br><br> for readability)
    print("\n" + "="*60)
    print("PREVIEW OF RESULTS")
    print("="*60)

    # Read and display the saved CSV
    with open(csv_file, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        row = next(reader)

        print("\n1. PLAIN LANGUAGE SUMMARY")
        print("-"*60)
        print(row['plain_summary'])

        print("\n2. STEELMAN FOR")
        print("-"*60)
        print(row['steelman_for'].replace("<br><br>", "\n\n"))

        print("\n3. STEELMAN AGAINST")
        print("-"*60)
        print(row['steelman_against'].replace("<br><br>", "\n\n"))

        print("\n4. VOTER DEBATE")
        print("-"*60)
        print(row['debate'].replace("<br><br>", "\n\n"))

        print("\n5. COMPREHENSION QUESTIONS")
        print("-"*60)
        for i in range(1, 6):
            print(f"\nQuestion {i}: {row[f'TF{i}']}")
            print(f"Answer: {row[f'TF{i}_ans']}")

Running approach 1: Plain summary...
Running approach 2: Voter debate...
Running approach 3a: Steelman FOR...
Running approach 3b: Steelman AGAINST...
Running approach 4: Comprehension questions...

Results saved to: ballot_analysis.csv

PREVIEW OF RESULTS

1. PLAIN LANGUAGE SUMMARY
------------------------------------------------------------
BACKGROUND
H.J.R.A No. A99 has been introduced to address the financial burden that ad valorem taxes impose on the agricultural retail sector, specifically for small business owners dealing in animal feed. Ad valorem taxes are based on the assessed value of tangible personal property, which includes animal feed held for sale. This measure aims to alleviate some of the financial strain on retailers by providing a tax exemption for this particular category of inventory. Supporters argue that such an exemption will help local businesses remain competitive, potentially lowering costs for consumers and boosting the local economy.

BALLOT MEASURE
The pr

In [18]:
import pandas as pd
df = pd.read_csv("../data/raw/ballot_September 29, 2025_15.16.csv")
df = df.iloc[2:]


In [22]:
df['pre_confident_1']

2     85
3     79
4     65
5     80
6    100
7    100
8     84
9     90
Name: pre_confident_1, dtype: object

Index(['StartDate', 'EndDate', 'Status', 'IPAddress', 'Progress',
       'Duration (in seconds)', 'Finished', 'RecordedDate', 'ResponseId',
       'RecipientLastName', 'RecipientFirstName', 'RecipientEmail',
       'ExternalReference', 'LocationLatitude', 'LocationLongitude',
       'DistributionChannel', 'UserLanguage', 'Q2', 'pid', 'would_vote_1',
       'pre_vote', 'pre_confident_1', 'pre_counter', 'pre_true_false_1',
       'pre_true_false_2', 'pre_true_false_3', 'pre_true_false_4',
       'pre_true_false_5', 'deb_would_vote_1', 'deb_vote', 'deb_confident_1',
       'deb_counter', 'deb_true_false_1', 'deb_true_false_2',
       'deb_true_false_3', 'deb_true_false_4', 'deb_true_false_5',
       'steel_would_vote_1', 'steel_vote', 'steel_confident_1',
       'steel_counter', 'steel_true_false_1', 'steel_true_false_2',
       'steel_true_false_3', 'steel_true_false_4', 'steel_true_false_5',
       'goal'],
      dtype='object')


switch
False    17
True      3
Name: count, dtype: int64

In [26]:
df_clean.query("diff_would_vote==-70").T

,18
pid,66318eecf4b2dd2836d910d8
pre_confident,85
pre_counter,5
pre_would_vote,35
pre_true_false_1,True
pre_true_false_2,False
pre_true_false_3,True
pre_true_false_4,False
pre_true_false_5,True
pre_truth_score,5


In [71]:
sorted(df.columns)

['DistributionChannel',
 'Duration (in seconds)',
 'EndDate',
 'ExternalReference',
 'Finished',
 'IPAddress',
 'LocationLatitude',
 'LocationLongitude',
 'Progress',
 'Q2',
 'RecipientEmail',
 'RecipientFirstName',
 'RecipientLastName',
 'RecordedDate',
 'ResponseId',
 'StartDate',
 'Status',
 'UserLanguage',
 'deb_confident_1',
 'deb_counter',
 'deb_true_false_1',
 'deb_true_false_2',
 'deb_true_false_3',
 'deb_true_false_4',
 'deb_true_false_5',
 'deb_vote',
 'deb_would_vote_1',
 'goal',
 'pid',
 'pre_confident_1',
 'pre_counter',
 'pre_true_false_1',
 'pre_true_false_2',
 'pre_true_false_3',
 'pre_true_false_4',
 'pre_true_false_5',
 'pre_vote',
 'steel_confident_1',
 'steel_counter',
 'steel_true_false_1',
 'steel_true_false_2',
 'steel_true_false_3',
 'steel_true_false_4',
 'steel_true_false_5',
 'steel_vote',
 'steel_would_vote_1',
 'would_vote_1']

In [5]:
df['pre_vote']

2     No
3    Yes
4    Yes
5    Yes
6    Yes
7    Yes
8    Yes
9    Yes
Name: pre_vote, dtype: object